In [98]:
import pandas as pd
import numpy as np
import pickle

## Loading the data

In [55]:
csv_file = "C:\\Users\kvba\\OneDrive - Ramboll\\Projects\\Ship traffic data of Fehmarnbelt\\data\\aisdk_20190102.csv"

In [61]:
%%time
ais_data = pd.read_csv(csv_file)
ais_data.head()

Wall time: 24.5 s


,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,...,Length,Type of position fixing device,Draught,Destination,ETA,Data source type,A,B,C,D
0,02/01/2019 00:00:00,Base Station,2194005,56.344267,4.272000,Unknown value,NaN,NaN,NaN,NaN,...,NaN,Surveyed,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
1,02/01/2019 00:00:00,Class B,266434000,57.715117,10.583545,Unknown value,NaN,0.0,NaN,NaN,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
2,02/01/2019 00:00:00,Class A,265522240,57.699495,11.666988,Unknown value,0.0,0.0,246.0,243.0,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
3,02/01/2019 00:00:00,Class A,231002000,91.000000,181.000000,Unknown value,NaN,NaN,NaN,NaN,...,138.0,GPS,5.6,FAYARD,26/12/2019 03:00:00,AIS,20.0,118.0,12.0,11.0
4,02/01/2019 00:00:00,Class A,219007401,57.122975,8.597812,Under way using engine,NaN,0.3,157.8,245.0,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN


This dataset takes about 25 seconds. Lets investigate the memory usage of the dataset:

In [5]:
ais_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9351207 entries, 0 to 9351206
Data columns (total 26 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   # Timestamp                     object 
 1   Type of mobile                  object 
 2   MMSI                            int64  
 3   Latitude                        float64
 4   Longitude                       float64
 5   Navigational status             object 
 6   ROT                             float64
 7   SOG                             float64
 8   COG                             float64
 9   Heading                         float64
 10  IMO                             object 
 11  Callsign                        object 
 12  Name                            object 
 13  Ship type                       object 
 14  Cargo type                      object 
 15  Width                           float64
 16  Length                          float64
 17  Type of position fixing dev

check the default dtypes in the dataframe:

## Data breakup

Lets see the data breakup amongst the columns in the given table. The total memory usage is 1.8 gbs:

In [60]:
total_mem = 0
for column in ais_data.columns:
    mem_usage = ais_data[column].memory_usage(index=False)/(1024**2)
    total_mem += mem_usage
    print("%s | %.2f MB" % (column, mem_usage))
print("Total memory (MB): %.2f" % total_mem)

# Timestamp | 71.34 MB
Type of mobile | 71.34 MB
MMSI | 71.34 MB
Latitude | 71.34 MB
Longitude | 71.34 MB
Navigational status | 71.34 MB
ROT | 71.34 MB
SOG | 71.34 MB
COG | 71.34 MB
Heading | 71.34 MB
IMO | 71.34 MB
Callsign | 71.34 MB
Name | 71.34 MB
Ship type | 71.34 MB
Cargo type | 71.34 MB
Width | 71.34 MB
Length | 71.34 MB
Type of position fixing device | 71.34 MB
Draught | 71.34 MB
Destination | 71.34 MB
ETA | 71.34 MB
Data source type | 71.34 MB
A | 71.34 MB
B | 71.34 MB
C | 71.34 MB
D | 71.34 MB
Total memory (MB): 1854.95


## Optimizing pandas dataframes

Idea is to downgrade the data types of the dataframe by looking at the maximum and minimum columns.

**for example**: A value with data type as int8 takes 8x times less memory compared to int64 data type.

<img src="https://miro.medium.com/max/715/1*4eMUql7kfw2Xdg_OCWVAuw.png"/>

For conversion - 
- convert the all float64 types to float16 except for coordinates to retain the precision
- convert all the string objects to pandas category dtype

## Numerical features

In [12]:
a = np.array([0.123456789123456789,2,3], dtype=np.float16)
print("16bit: ", a[0])

a = np.array([0.123456789123456789,2,3], dtype=np.float32)
print("32bit: ", a[0])

b = np.array([0.123456789123456789,2,3], dtype=np.float64)
print("64bit: ", b[0])

16bit:  0.1235
32bit:  0.12345679
64bit:  0.12345678912345678


Heading is store as float but represents  integer value ranging from 0-510.
We can convert the values to int16

In [8]:
x1 = ais_data["Heading"].memory_usage(index=False, deep=True)
x2 = df["Heading"].astype("float16").memory_usage(index=False, deep=True)
print("Default the memory usage is: %.2f MB" % (x1/(1024**2)))
print("After conversion the memory usage is: %.2f MB" % (x2/(1024**2)))
(x1-x2)/x1*100

Default the memory usage is: 71.34 MB
After conversion the memory usage is: 17.84 MB


75.0

Similarly we can convert this for ROT, COG, speeds, height, width, draught, A, B, C, D:

In [9]:
float16_cols = ais_data.dtypes[ais_data.dtypes == "float64"][2:]
float16_cols

ROT        float64
SOG        float64
COG        float64
Heading    float64
Width      float64
Length     float64
Draught    float64
A          float64
B          float64
C          float64
D          float64
dtype: object

In [10]:
print("Column | Min value | Max val | % missing")
for cols in float16_cols.index:
    min_val, max_val = ais_data[cols].min(), ais_data[cols].max()
    null_values = ais_data[cols].isnull().sum()/ len(ais_data[cols])*100
    print("%s | %.2f | %.2f | %.2f" % (cols, min_val, max_val, null_values))

Column | Min value | Max val | % missing
ROT | -708.70 | 708.70 | 34.13
SOG | 0.00 | 179.00 | 6.64
COG | 0.00 | 359.90 | 9.94
Heading | 0.00 | 510.00 | 21.73
Width | 1.00 | 125.00 | 10.25
Length | 2.00 | 405.00 | 10.16
Draught | 0.10 | 25.50 | 25.17
A | 1.00 | 260.00 | 10.39
B | 1.00 | 335.00 | 10.82
C | 1.00 | 63.00 | 10.97
D | 1.00 | 62.00 | 11.23


- We can easily convert these above values to a lower dtype such as int or float since the range is low
- Converting most of these values to integer would be efficient but since almost all the columns have null values, it cannot be converted to int
- Better it is to convert the float64 dtype values to float16, which can accomodate nan values as well. 

In [11]:
total_mem = 0
new_mem = 0

for col in float16_cols.index:
    x1 = ais_data[col].memory_usage(index=False, deep=True)
    x2 = df[col].astype("float16").memory_usage(index=False, deep=True)
    total_mem += x1
    new_mem += x2

print("Default the memory usage is: %.2f MB" % (total_mem/(1024**2)))
print("After conversion the memory usage is: %.2f MB" % (new_mem/(1024**2)))
print("Total of %.f %% reduction in the storage" % ((total_mem-new_mem)/total_mem*100))

Default the memory usage is: 784.78 MB
After conversion the memory usage is: 196.20 MB
Total of 75 % reduction in the storage


## Categorical data

Converting the string data type to categorical types. Removing timestamp and ETA as they are datetime objects

In [125]:
print("Column | % missing")
string_cols = ais_data.dtypes[ais_data.dtypes == "object"].drop(["# Timestamp", "ETA"])

for cols in string_cols.index:
    null_values = ais_data[cols].isnull().sum()/ len(ais_data[cols])*100
    print("%s | %.2f |" % (cols, null_values))

Column | % missing
Type of mobile | 0.00 |
Navigational status | 0.00 |
IMO | 0.00 |
Callsign | 7.41 |
Name | 5.94 |
Ship type | 0.00 |
Cargo type | 84.99 |
Type of position fixing device | 0.00 |
Destination | 29.05 |
Data source type | 0.00 |


Note: when converting the string objects to categorical, the missing values will be stored as nan

In [118]:
def mem_reduction(df, col="Type of mobile", convert_to="category"):
    x1 = df[col].memory_usage(index=False, deep=True)
    x2 = df[col].astype(convert_to).memory_usage(index=False, deep=True)
    print("Default the memory usage is: %.2f MB" % (x1/(1024**2)))
    print("After conversion the memory usage is: %.2f MB" % (x2/(1024**2)))
    print("Total %.2f %% reduction in memory storage" % ((x1-x2)/x1*100))

In [63]:
df["Type of mobile"].value_counts()

Class A                          8573959
Base Station                      446100
Class B                           261495
AtoN                               68015
SAR Airborne                        1632
Search and Rescue Transponder          6
Name: Type of mobile, dtype: int64

Since this is a catergorical type of data, we can restore the values as categorical values:

In [64]:
df["Type of mobile"].astype("category").sample(4)

899025     Class A
7217377    Class A
3686460    Class A
2599428    Class A
Name: Type of mobile, dtype: category
Categories (6, object): ['AtoN', 'Base Station', 'Class A', 'Class B', 'SAR Airborne', 'Search and Rescue Transponder']

In [119]:
mem_reduction(ais_data, col="Type of mobile", convert_to="category")

Default the memory usage is: 572.69 MB
After conversion the memory usage is: 8.92 MB
Total 98.44 % reduction in memory storage


In [67]:
string_cols = ais_data.dtypes[ais_data.dtypes == "object"]
string_cols

# Timestamp                       object
Type of mobile                    object
Navigational status               object
IMO                               object
Callsign                          object
Name                              object
Ship type                         object
Cargo type                        object
Type of position fixing device    object
Destination                       object
ETA                               object
Data source type                  object
dtype: object

Here only few columns can be converted to categorical type, like:
- Type of mobile
- Navigational status
- Type of position fixing device
- Data source type


Here ETA and # Timestamp shall be read as datetime objects

In [68]:
ais_data["Navigational status"].value_counts()

Under way using engine                 5462606
Engaged in fishing                     1516496
Unknown value                          1472244
Moored                                  328509
Under way sailing                       164395
Restricted maneuverability              157223
At anchor                               101167
Constrained by her draught               70928
Reserved for future amendment [HSC]      39985
Not under command                        16972
Reserved for future amendment [WIG]      11614
Reserved for future use [11]              7533
Aground                                   1532
Reserved for future use [12]                 2
AIS-SART                                     1
Name: Navigational status, dtype: int64

In [120]:
mem_reduction(ais_data, col="Navigational status", convert_to="category")

Default the memory usage is: 680.50 MB
After conversion the memory usage is: 8.92 MB
Total 98.69 % reduction in memory storage


In [121]:
mem_reduction(ais_data, col="Callsign", convert_to="category")

Default the memory usage is: 531.56 MB
After conversion the memory usage is: 18.04 MB
Total 96.61 % reduction in memory storage


Lets see which object dtype we can convert to categorical dtype:

- we have to keep in mind about the nunique values for each columns
- if the values are say less than 10% of the total ais data, it is wise to convert it to categorial
- Since some of the time nunique values are above 80%, in that case storing string columns as categorical dtype will end up using more memory

In [126]:
for col in string_cols.index:
    unique_values = ais_data[col].nunique()
    print("'%s' has about %.2f %% unique values" % (col, unique_values/len(ais_data)*100))

'Type of mobile' has about 0.00 % unique values
'Navigational status' has about 0.00 % unique values
'IMO' has about 0.01 % unique values
'Callsign' has about 0.02 % unique values
'Name' has about 0.02 % unique values
'Ship type' has about 0.00 % unique values
'Cargo type' has about 0.00 % unique values
'Type of position fixing device' has about 0.00 % unique values
'Destination' has about 0.01 % unique values
'Data source type' has about 0.00 % unique values


Here we can see that - 
- ~~call *IMO, sign, name & destination* are some columns which are having a lot of unique values. Hence they cannot be converted to category.~~
- since the unqiue values in all the columns are negligible compared to the total data, we can convert these to categorical dtype
- also ETA is timestamp, we need to convert it to datetime

In [128]:
for col in string_cols.index:
    print(col)
    mem_reduction(ais_data, col=col, convert_to="category")
    print("-"*30)

Type of mobile
Default the memory usage is: 572.69 MB
After conversion the memory usage is: 8.92 MB
Total 98.44 % reduction in memory storage
------------------------------
Navigational status
Default the memory usage is: 680.50 MB
After conversion the memory usage is: 8.92 MB
Total 98.69 % reduction in memory storage
------------------------------
IMO
Default the memory usage is: 570.79 MB
After conversion the memory usage is: 17.94 MB
Total 96.86 % reduction in memory storage
------------------------------
Callsign
Default the memory usage is: 531.56 MB
After conversion the memory usage is: 18.04 MB
Total 96.61 % reduction in memory storage
------------------------------
Name
Default the memory usage is: 584.35 MB
After conversion the memory usage is: 18.06 MB
Total 96.91 % reduction in memory storage
------------------------------
Ship type
Default the memory usage is: 565.95 MB
After conversion the memory usage is: 8.92 MB
Total 98.42 % reduction in memory storage
-----------------

## Loading the csv using typecasting

In [129]:
list(string_cols.index)

['Type of mobile',
 'Navigational status',
 'IMO',
 'Callsign',
 'Name',
 'Ship type',
 'Cargo type',
 'Type of position fixing device',
 'Destination',
 'Data source type']

In [85]:
list(float16_cols.index)

['ROT',
 'SOG',
 'COG',
 'Heading',
 'Width',
 'Length',
 'Draught',
 'A',
 'B',
 'C',
 'D']

Convert the dtypes to the optimized dtypes for each columns

In [130]:
%%time
dtype_dict = {
    "MMSI" : "int32",
    'ROT' : "float16",
    'SOG' : "float16",
    'COG' : "float16",
    'Heading' : "float16",
    'Width' : "float16",
    'Length' : "float16",
    'Draught' : "float16",
    'A' : "float16",
    'B' : "float16",
    'C' : "float16",
    'D' : "float16",
    'Type of mobile' : "category",
    'Navigational status' : "category",
    'IMO' : "category",
    'Callsign' : "category",
    'Name' : "category",
    'Ship type' : "category",
    'Cargo type' : "category",
    'Type of position fixing device' : "category",
    'Destination' : "category",
    'Data source type' : "category"
}

df = pd.read_csv(csv_file, dtype=dtype_dict, parse_dates=["# Timestamp", "ETA"])

Wall time: 38.7 s


In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9351207 entries, 0 to 9351206
Data columns (total 26 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   # Timestamp                     datetime64[ns]
 1   Type of mobile                  category      
 2   MMSI                            int32         
 3   Latitude                        float64       
 4   Longitude                       float64       
 5   Navigational status             category      
 6   ROT                             float16       
 7   SOG                             float16       
 8   COG                             float16       
 9   Heading                         float16       
 10  IMO                             category      
 11  Callsign                        category      
 12  Name                            category      
 13  Ship type                       category      
 14  Cargo type                      category      
 15

In [144]:
original = ais_data.memory_usage(index=False).sum()/(1024**3)
optimized = df.memory_usage(index=False).sum()/(1024**3)
print("The original file was around %.2f gbs whereas optimized one was %.2f gbs." % (original, optimized))
print("The total reduction was around %.2f %% in memory usage." % (((original-optimized)/original)*100))

The original file was around 1.81 gbs whereas optimized one was 0.63 gbs
The total reduction was around 65.37 % in memory usage


## Exporting and importing as pickle file

In [132]:
%%time
with open("aisdk_20190102.pickle", "wb") as pkl_file:
    pickle.dump(df, pkl_file)

Wall time: 1.89 s


In [133]:
%%time
with open("aisdk_20190102.pickle", "rb") as pkl_file:
    data = pickle.load(pkl_file)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9351207 entries, 0 to 9351206
Data columns (total 26 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   # Timestamp                     datetime64[ns]
 1   Type of mobile                  category      
 2   MMSI                            int32         
 3   Latitude                        float64       
 4   Longitude                       float64       
 5   Navigational status             category      
 6   ROT                             float16       
 7   SOG                             float16       
 8   COG                             float16       
 9   Heading                         float16       
 10  IMO                             category      
 11  Callsign                        category      
 12  Name                            category      
 13  Ship type                       category      
 14  Cargo type                      category      
 15

## Conclusion

- If we convert the dataframe dtypes to a downgrade version, we reduce the memory usage more than 60% the original. 
- Saving the objects types which were originally using 8bytes of memory can be optimized to pandas categorical dtype, which uses hash tables to encode the unique values
- However not always the conversion of object to categorical is recommended, it depends upon the number of unique values of a column. 
- In this case we can convert all the object type to categorical since unique values were fractional
- Converting the float64 dtype to float16 can efficiently reduce the memory usuage.
- All conversion can significantly reduce the memory usage of the file. 
- Exporting the file to pickle file and reading the back the pickle file while loading can significanly reduce the reading time compared to pandas read csv.


We can use this pipeline to store all the csv files to pickle files and use it for further processing

**Reference:**

1. [Optimize the pandas dataframe](https://towardsdatascience.com/optimize-pandas-memory-usage-while-reading-large-datasets-1b047c762c9b)
2. [More on pandas categorical dtype and how it works](https://medium.com/analytics-vidhya/unleash-the-power-of-pandas-category-dtype-encode-categorical-data-in-smarter-ways-eb787cd274df)